In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from keras.optimizers import Adam

from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_absolute_error

In [2]:
params = {
    'epochs': 200,
    'test_size': 0.2,
    'random_state': 1337,
    'validation_split': 0.2
}

In [3]:
df = pd.read_csv("../data/qsc_out.random_scan_nfp2.csv")

In [4]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,y0,y1,y2,y3,y4,y5,y6
0,-0.104856,0.197790,0.001260,-0.011792,0.000794,0.000120,-0.687180,-0.215177,0.948834,0.264545,0.404154,0.631039,0.898928,1.581192,0.334374
1,-0.110385,0.188144,0.001381,-0.011358,0.000855,0.000138,-0.658897,-0.233486,0.930837,0.298531,0.469811,0.639501,0.876879,1.568947,0.336379
2,-0.095042,0.132051,0.000675,-0.008909,0.000477,0.000348,-0.671871,-0.944641,1.526347,0.385624,0.334515,0.468280,0.804239,1.288313,0.331086
3,-0.115912,-0.207162,0.001411,0.012060,0.000871,-0.000108,-0.736734,0.012462,0.783335,0.278748,0.497138,0.645087,0.926717,1.717088,0.338459
4,-0.096390,-0.217079,0.000933,0.010517,0.000621,-0.000126,-0.764525,-0.048433,0.925351,0.272636,0.548743,0.631384,0.941509,1.549327,0.331432


In [5]:
df.shape

(29674, 15)

In [6]:
x_columns = [col for col in df.columns if col.startswith('x')]
y_columns = [col for col in df.columns if col.startswith('y')]

Y = df[y_columns].values
X = df[x_columns].values

In [7]:
def preprocess_data(X_train, X_test, Y_train, Y_test, params):
    scaler_x = StandardScaler().fit(X_train)
    scaler_y = StandardScaler().fit(Y_train)
    X_train = scaler_x.transform(X_train)
    X_test = scaler_x.transform(X_test)
    Y_train = scaler_y.transform(Y_train)
    Y_test = scaler_y.transform(Y_test)

    input_shape = X_train.shape[1]
    
    output_shape = Y_train.shape[1]
    return X_train, X_test, Y_train, Y_test, input_shape, output_shape, scaler_x, scaler_y

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=params['test_size'], 
                                                    random_state=params['random_state'])

X_train, X_test, Y_train, Y_test, input_shape, output_shape, scaler_x, scaler_y = preprocess_data(X_train, X_test, Y_train, Y_test, params)

In [8]:
X_train.mean(axis=0), X_train.std(axis=0) 

(array([-1.86603895e-17, -2.39778990e-17, -4.12750044e-17, -8.97978632e-17,
         1.45237693e-17,  4.88584184e-17, -7.02959790e-15, -1.74744727e-15]),
 array([1., 1., 1., 1., 1., 1., 1., 1.]))

## Appears to be drift, perhaps the sample is not big enough

In [9]:
X_test.mean(axis=0), X_test.std(axis=0) 

(array([ 0.01621399, -0.00112999,  0.01755134,  0.01955565,  0.01673425,
         0.00871709, -0.01095845, -0.02030808]),
 array([0.99488322, 0.99650294, 0.78686316, 0.96749519, 0.93053514,
        0.90833692, 1.01170468, 1.00410648]))

In [10]:
Y_train.mean(axis=0), Y_train.std(axis=0) 

(array([-1.23324421e-14,  2.75938148e-14, -7.59591967e-15, -1.74703992e-14,
        -3.12392412e-14, -5.68256506e-14, -1.97065971e-13]),
 array([1., 1., 1., 1., 1., 1., 1.]))

In [11]:
Y_test.mean(axis=0), Y_test.std(axis=0)

(array([ 0.002863  ,  0.0093879 , -0.00576779, -0.01150026, -0.01160122,
        -0.01466698, -0.0055547 ]),
 array([1.00800321, 1.00616058, 1.0009353 , 0.99290648, 0.99481008,
        1.00920012, 0.71155792]))

## Dummy regressor

In [12]:
from sklearn.dummy import DummyRegressor

In [13]:
regr = MultiOutputRegressor(DummyRegressor(strategy="mean")).fit(X_train, Y_train)
regr.predict(X_train)

array([[-1.23299623e-14,  2.76159547e-14, -7.59899700e-15, ...,
        -3.12143139e-14, -5.68119908e-14, -1.97071672e-13],
       [-1.23299623e-14,  2.76159547e-14, -7.59899700e-15, ...,
        -3.12143139e-14, -5.68119908e-14, -1.97071672e-13],
       [-1.23299623e-14,  2.76159547e-14, -7.59899700e-15, ...,
        -3.12143139e-14, -5.68119908e-14, -1.97071672e-13],
       ...,
       [-1.23299623e-14,  2.76159547e-14, -7.59899700e-15, ...,
        -3.12143139e-14, -5.68119908e-14, -1.97071672e-13],
       [-1.23299623e-14,  2.76159547e-14, -7.59899700e-15, ...,
        -3.12143139e-14, -5.68119908e-14, -1.97071672e-13],
       [-1.23299623e-14,  2.76159547e-14, -7.59899700e-15, ...,
        -3.12143139e-14, -5.68119908e-14, -1.97071672e-13]])

In [14]:
mean_absolute_error(Y_train, regr.predict(X_train))

0.7819301538758372

In [15]:
mean_absolute_error(Y_test, regr.predict(X_test))

0.7826444903716244

## Train a linear regression for debugging

In [16]:
regr = MultiOutputRegressor(Ridge(random_state=123)).fit(X_train, Y_train)
regr.predict(X_train)

array([[-0.01118562, -0.25258042, -0.07693146, ...,  0.15934534,
         0.02889025, -0.02566576],
       [ 0.33075672,  0.74101837,  0.27335388, ..., -0.3718137 ,
        -0.14318812, -0.4488022 ],
       [-0.22386175, -0.38369246, -0.11028056, ...,  0.22433786,
         0.10529634,  0.09333019],
       ...,
       [ 0.01024576, -0.05781049, -0.02785113, ..., -0.06734454,
        -0.04966926,  0.16428362],
       [-0.09533647, -0.20393649, -0.04846468, ...,  0.42137434,
         0.14091655,  1.00339397],
       [-0.10952675, -0.16497835, -0.03597803, ...,  0.16906614,
         0.06585663,  0.100133  ]])

In [17]:
mean_absolute_error(Y_train, regr.predict(X_train))

0.7420778534372884

In [18]:
mean_absolute_error(Y_test, regr.predict(X_test))

0.7453412087802475

## Simplest neural network

In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [20]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(input_shape, activation="relu", name="layer_in"),
        layers.Dense(128, activation="relu", name="layer2"),
        layers.Dense(64, activation="relu", name="layer3"),
        layers.Dense(output_shape, name="layer_out"),
    ]
)

model(X_train[0:1])

<tf.Tensor: shape=(1, 7), dtype=float32, numpy=
array([[ 0.14221711,  0.10756896, -0.03302422, -0.06281132,  0.13048202,
        -0.09029931,  0.06472876]], dtype=float32)>

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_in (Dense)            (1, 8)                    72        
                                                                 
 layer2 (Dense)              (1, 128)                  1152      
                                                                 
 layer3 (Dense)              (1, 64)                   8256      
                                                                 
 layer_out (Dense)           (1, 7)                    455       
                                                                 
Total params: 9935 (38.81 KB)
Trainable params: 9935 (38.81 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanAbsoluteError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanAbsoluteError()],
)

In [23]:
print("Fit model on training data")
history = model.fit(
    X_train,
    Y_train,
    batch_size=64,
    epochs=200,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(X_test, Y_test),
)

Fit model on training data
Epoch 1/200
371/371 [==============================] - 0s 742us/step - loss: 0.6695 - mean_absolute_error: 0.6695 - val_loss: 0.6094 - val_mean_absolute_error: 0.6094
Epoch 2/200
371/371 [==============================] - 1s 2ms/step - loss: 0.5678 - mean_absolute_error: 0.5678 - val_loss: 0.5269 - val_mean_absolute_error: 0.5269
Epoch 3/200
371/371 [==============================] - 0s 570us/step - loss: 0.5007 - mean_absolute_error: 0.5007 - val_loss: 0.4762 - val_mean_absolute_error: 0.4762
Epoch 4/200
371/371 [==============================] - 0s 572us/step - loss: 0.4581 - mean_absolute_error: 0.4581 - val_loss: 0.4402 - val_mean_absolute_error: 0.4402
Epoch 5/200
371/371 [==============================] - 0s 570us/step - loss: 0.4309 - mean_absolute_error: 0.4309 - val_loss: 0.4239 - val_mean_absolute_error: 0.4239
Epoch 6/200
371/371 [==============================] - 0s 567us/step - loss: 0.4120 - mean_absolute_error: 0.4120 - val_loss: 0.3926 - val_m

Epoch 50/200
371/371 [==============================] - 0s 593us/step - loss: 0.2136 - mean_absolute_error: 0.2136 - val_loss: 0.2092 - val_mean_absolute_error: 0.2092
Epoch 51/200
371/371 [==============================] - 0s 596us/step - loss: 0.2118 - mean_absolute_error: 0.2118 - val_loss: 0.2118 - val_mean_absolute_error: 0.2118
Epoch 52/200
371/371 [==============================] - 0s 585us/step - loss: 0.2110 - mean_absolute_error: 0.2110 - val_loss: 0.2028 - val_mean_absolute_error: 0.2028
Epoch 53/200
371/371 [==============================] - 0s 587us/step - loss: 0.2102 - mean_absolute_error: 0.2102 - val_loss: 0.2014 - val_mean_absolute_error: 0.2014
Epoch 54/200
371/371 [==============================] - 0s 592us/step - loss: 0.2087 - mean_absolute_error: 0.2087 - val_loss: 0.2008 - val_mean_absolute_error: 0.2008
Epoch 55/200
371/371 [==============================] - 0s 607us/step - loss: 0.2079 - mean_absolute_error: 0.2079 - val_loss: 0.2245 - val_mean_absolute_error:

Epoch 99/200
371/371 [==============================] - 0s 595us/step - loss: 0.1699 - mean_absolute_error: 0.1699 - val_loss: 0.1796 - val_mean_absolute_error: 0.1796
Epoch 100/200
371/371 [==============================] - 0s 683us/step - loss: 0.1693 - mean_absolute_error: 0.1693 - val_loss: 0.1639 - val_mean_absolute_error: 0.1639
Epoch 101/200
371/371 [==============================] - 0s 654us/step - loss: 0.1684 - mean_absolute_error: 0.1684 - val_loss: 0.1693 - val_mean_absolute_error: 0.1693
Epoch 102/200
371/371 [==============================] - 0s 626us/step - loss: 0.1682 - mean_absolute_error: 0.1682 - val_loss: 0.1634 - val_mean_absolute_error: 0.1634
Epoch 103/200
371/371 [==============================] - 0s 599us/step - loss: 0.1681 - mean_absolute_error: 0.1681 - val_loss: 0.1696 - val_mean_absolute_error: 0.1696
Epoch 104/200
371/371 [==============================] - 0s 601us/step - loss: 0.1674 - mean_absolute_error: 0.1674 - val_loss: 0.1705 - val_mean_absolute_e

371/371 [==============================] - 0s 605us/step - loss: 0.1524 - mean_absolute_error: 0.1524 - val_loss: 0.1631 - val_mean_absolute_error: 0.1631
Epoch 148/200
371/371 [==============================] - 0s 635us/step - loss: 0.1523 - mean_absolute_error: 0.1523 - val_loss: 0.1501 - val_mean_absolute_error: 0.1501
Epoch 149/200
371/371 [==============================] - 0s 607us/step - loss: 0.1520 - mean_absolute_error: 0.1520 - val_loss: 0.1552 - val_mean_absolute_error: 0.1552
Epoch 150/200
371/371 [==============================] - 0s 608us/step - loss: 0.1520 - mean_absolute_error: 0.1520 - val_loss: 0.1511 - val_mean_absolute_error: 0.1511
Epoch 151/200
371/371 [==============================] - 0s 589us/step - loss: 0.1518 - mean_absolute_error: 0.1518 - val_loss: 0.1577 - val_mean_absolute_error: 0.1577
Epoch 152/200
371/371 [==============================] - 0s 622us/step - loss: 0.1519 - mean_absolute_error: 0.1519 - val_loss: 0.1495 - val_mean_absolute_error: 0.1495


371/371 [==============================] - 0s 630us/step - loss: 0.1426 - mean_absolute_error: 0.1426 - val_loss: 0.1459 - val_mean_absolute_error: 0.1459
Epoch 196/200
371/371 [==============================] - 0s 584us/step - loss: 0.1430 - mean_absolute_error: 0.1430 - val_loss: 0.1464 - val_mean_absolute_error: 0.1464
Epoch 197/200
371/371 [==============================] - 0s 597us/step - loss: 0.1427 - mean_absolute_error: 0.1427 - val_loss: 0.1492 - val_mean_absolute_error: 0.1492
Epoch 198/200
371/371 [==============================] - 0s 588us/step - loss: 0.1427 - mean_absolute_error: 0.1427 - val_loss: 0.1580 - val_mean_absolute_error: 0.1580
Epoch 199/200
371/371 [==============================] - 0s 589us/step - loss: 0.1424 - mean_absolute_error: 0.1424 - val_loss: 0.1489 - val_mean_absolute_error: 0.1489
Epoch 200/200
371/371 [==============================] - 0s 583us/step - loss: 0.1419 - mean_absolute_error: 0.1419 - val_loss: 0.1428 - val_mean_absolute_error: 0.1428


In [24]:
mean_absolute_error(Y_train, model.predict(X_train))

742/742 [==============================] - 0s 311us/step


0.13744080010353865

In [25]:
mean_absolute_error(Y_test, model.predict(X_test))

186/186 [==============================] - 0s 321us/step


0.14277609028311838